<h1><center><font size="6">Predicting Energy Consumption </font></center></h1>

<h2><center><font size="4">Dataset used:Kaggle </font></center></h2>

<h3><center><font size="4"> https://datahack.analyticsvidhya.com/contest/janatahack-time-series-forecasting/True/</font></center></h2>

<a id="top"></a> <br>
## Table of  Contents
1. [Introduction](#1)

1. [Initialization](#2)
    1. [Load packages](#21)
    1. [Define Metadata](#22)
    
1. [Load Data](#3)

1. [Data Insights](#4)
    1. [Data Structure](#41)
    1. [Summary Stats](#42)
    1. [Unique Value Checking](#43)
    1. [Identifying 'Bad Columns'](#44)

1. [Data Cleansing](#5)
    1. [Data Reduction](#51)
        1. [Dropping Bad Columns](#511)
        1. [Null Value Removal](#512)
    1. [Data Type Conversion](#52)

1. [Modelling Workflow](#6)
    1. [Data Prep](#61)
        1. [Categorical to Ordinal Encoding](#611)
        1. [Normalizing Numerical Variables](#612)
        1. [Feature Target Split](#613)
        1. [Train-Test Split](#614)
    1. [Estimate of Baseline Accuracy - Class Distributions](#62)
    1. [Predictive Modelling](#63)
        1. [ML Model Fitting](#631)
        1. [Model Performance Metrics Report](#632)
    1. [Performance Metrics for Imbalanced Datasets](#64)
    
1. [References](#7)

# <a id='1'>Introduction</a>  

Company of Electrolysia supplies electricity to the city. It is looking to optimise its electricity production based on the historical electricity consumption of the people of Electrovania. 
 
The company has hired you as a Data Scientist to investigate the past consumption and the weather information to come up with a model that catches the trend as accurately as possible. You have to bear in mind that there are many factors that affect electricity consumption and not all can be measured. Electrolysia has provided you this data on hourly data spanning five years. 
 
For this competition, the training set is comprised of the first 23 days of each month and the test set is the 24th to the end of the month, where the public leaderboard is based on the first two days of test, whereas the private leaderboard considers the rest of the days. Your task is to predict the electricity consumption on hourly basis.

## <a id='2'>Initialization</a>  


### <a id='21'>Load Packages</a>  

Load the minimum number of packages to get started and add more as we go along

In [1]:
import pandas as pd 
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

import warnings
warnings.filterwarnings('ignore')
# This is to suppress any deprecation warnings 

import os

import sys
sys.path.append("/home/paperspace/Kaggle Grandmaster/Udemy Superstar/Classification") # Adds higher directory to python modules path.


### <a id='22'>Define Metadata</a>  

In [2]:
# Define the name of the target class column here instead of manually typing it out everywhere
target_class_name = 'electricity_consumption'

input_file_name = 'train_6BJx641.csv'
sample_submission_file_name = 'sample_submission_bYgKb77.csv'
test_file_name = 'test_pavJagI.csv'

intermediate_data_path = '../intermediate data/'

output_df_file_name = 'day1_cleaned_df_output_' 

# Enter the date in which this notebook is run
export_date = '2020502'

In [3]:
# Check ou tthe names of the files in the input folder that we need to load as dataframe
import os
print(os.listdir('input'))
# This will list all the files in this folder

['sample_submission_bYgKb77.csv', 'train_6BJx641.csv', 'test_pavJagI.csv']


## <a id='3'>Load Data</a>  

In [4]:
df = pd.read_csv('input/' + input_file_name, index_col='ID')
df.sample(5)

,datetime,temperature,var1,pressure,windspeed,var2,electricity_consumption
ID,,,,,,,
542,2013-07-23 14:00:00,-5.0,-18.6,992.0,193.100,A,192.0
19107,2015-09-05 03:00:00,-5.7,-5.7,984.0,3.405,A,456.0
6851,2014-04-12 11:00:00,8.6,3.6,975.0,3.665,A,192.0
27096,2016-08-03 00:00:00,-2.9,-26.4,993.0,28.695,A,195.0
18747,2015-08-21 03:00:00,-8.6,-12.9,1007.0,47.110,A,264.0


Lets check the head & tail to make sure there is nothing going on

In [5]:
df.head(3)
# What you are checking here is whether there are some unwanted headers from teh input file that 
# can corrupt our data
# If found, either open the file and manually delete it or only load the relevant columns

# You can see the Target column is actually upfront
# There are 120 features even before we combine the other datasets with this

,datetime,temperature,var1,pressure,windspeed,var2,electricity_consumption
ID,,,,,,,
0,2013-07-01 00:00:00,-11.4,-17.1,1003.0,571.910,A,216.0
1,2013-07-01 01:00:00,-12.1,-19.3,996.0,575.040,A,210.0
2,2013-07-01 02:00:00,-12.9,-20.0,1000.0,578.435,A,225.0


In [6]:
df.tail(3)
# similar logic here

,datetime,temperature,var1,pressure,windspeed,var2,electricity_consumption
ID,,,,,,,
34893,2017-06-23 21:00:00,-1.4,-12.9,995.0,61.275,A,213.0
34894,2017-06-23 22:00:00,-2.9,-11.4,996.0,67.210,A,210.0
34895,2017-06-23 23:00:00,-2.1,-11.4,1009.0,71.880,A,210.0


No trouble with loading the data. Both the head and tail are clean

In [7]:
# # Are the number of IDs same as the number of rows?
# len(df['ID'].unique())

In [8]:
test_df = pd.read_csv('input/' + test_file_name, index_col='ID')
test_df.sample(5)

,datetime,temperature,var1,pressure,windspeed,var2
ID,,,,,,
15173,2015-03-25 05:00:00,6.4,5.0,983.0,3.605,A
654,2013-07-28 06:00:00,-11.4,-18.6,1002.0,32.815,A
20308,2015-10-25 04:00:00,3.6,-7.9,977.0,2.640,A
696,2013-07-30 00:00:00,-7.1,-17.9,1012.0,83.760,A
22613,2016-01-29 05:00:00,11.4,9.3,963.0,11.530,A


In [9]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8568 entries, 552 to 35063
Data columns (total 6 columns):
datetime       8568 non-null object
temperature    8568 non-null float64
var1           8568 non-null float64
pressure       8568 non-null float64
windspeed      8568 non-null float64
var2           8568 non-null object
dtypes: float64(4), object(2)
memory usage: 468.6+ KB


In [10]:
# train_unique_id_list = df['ID'].unique()
# test_unique_id_list = test_df['ID'].unique()
# len(set(train_unique_id_list) - set(test_unique_id_list))
# # As expected there is nothing funny going on with data leakage. All the IDs in test are unique

## <a id=4 > Data Insights

### <a id='41'>Data Structure</a> 

In [11]:
# Lets see the data structure
df.info()
# This will give you how many rows & columns are there

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26496 entries, 0 to 34895
Data columns (total 7 columns):
datetime                   26496 non-null object
temperature                26496 non-null float64
var1                       26496 non-null float64
pressure                   26496 non-null float64
windspeed                  26496 non-null float64
var2                       26496 non-null object
electricity_consumption    26496 non-null float64
dtypes: float64(5), object(2)
memory usage: 1.6+ MB


Seems the data has been preprocessed to the point where all the variables are being recognized as floats

### <a id='42'>Summary Stats</a> 

In [12]:
df.describe()
# check out each column's summary statistics to and infer and have follow on visualizations 
# Note that only the numerical columns will be described. Make sure to ignore non-numerical columns
# like 'customer_id' before you describe
# Also you will want to exclude the integer columns

,temperature,var1,pressure,windspeed,electricity_consumption
count,26496.000000,26496.000000,26496.000000,26496.000000,26496.000000
mean,5.098989,-1.916233,986.450615,23.959956,298.359601
std,8.682860,10.424860,12.002647,48.280321,108.020555
min,-17.100000,-32.900000,953.000000,1.075000,174.000000
25%,-2.900000,-10.700000,978.000000,3.155000,219.000000
50%,6.400000,-1.400000,986.000000,6.545000,267.000000
75%,12.100000,7.900000,995.000000,22.260000,342.000000
max,23.600000,18.600000,1024.000000,586.600000,1386.000000


### <a id='43'>Unique Value Checking</a> 

In [13]:
for column in df.columns:
    print(column, len(df[column].unique()))

datetime 26496
temperature 60
var1 71
pressure 73
windspeed 5603
var2 3
electricity_consumption 252


In [14]:
# The unique values in electricity consumption are surprisingly few. 
# The baseline submission will be with just the median value of 267

### <a id='44'>Identifying Bad Columns</a> 

In [15]:
from helper_functions_comprehensive import find_bad_columns_function

null_col_list, blank_space_col_list, constant_numeric_col_list, \
constant_non_numeric_col_list, duplicate_col_list = find_bad_columns_function(df)

Identified 0 features with atleast one null
Identified 0 features with atleast one blank space
Identified 0 numeric features that have quasi-constant values
Identified 0 non-numeric features that have quasi-constant values
Identified 0 features that are duplicates of other features


In [16]:
# Surprisingly clean data

## <a id='5'>Data Cleansing</a> 

### <a id='53'>Data Type Conversion</a> 

#### <a id='532'>Data Encoding</a> 

In [17]:
# You cant use LabelEncoder because it will give an implicit ordering
# Use get dummies to create one hot encoded columns and drop one of the columns to avoid collinearity
encoded_df = pd.get_dummies(df['var2'],prefix='var2', drop_first=True)
df = pd.concat([df,encoded_df], axis=1)
df.head()

,datetime,temperature,var1,pressure,windspeed,var2,electricity_consumption,var2_B,var2_C
ID,,,,,,,,,
0,2013-07-01 00:00:00,-11.4,-17.1,1003.0,571.910,A,216.0,0,0
1,2013-07-01 01:00:00,-12.1,-19.3,996.0,575.040,A,210.0,0,0
2,2013-07-01 02:00:00,-12.9,-20.0,1000.0,578.435,A,225.0,0,0
3,2013-07-01 03:00:00,-11.4,-17.1,995.0,582.580,A,216.0,0,0
4,2013-07-01 04:00:00,-11.4,-19.3,1005.0,586.600,A,222.0,0,0


In [19]:
del df['var2']
df.head()

,datetime,temperature,var1,pressure,windspeed,electricity_consumption,var2_B,var2_C
ID,,,,,,,,
0,2013-07-01 00:00:00,-11.4,-17.1,1003.0,571.910,216.0,0,0
1,2013-07-01 01:00:00,-12.1,-19.3,996.0,575.040,210.0,0,0
2,2013-07-01 02:00:00,-12.9,-20.0,1000.0,578.435,225.0,0,0
3,2013-07-01 03:00:00,-11.4,-17.1,995.0,582.580,216.0,0,0
4,2013-07-01 04:00:00,-11.4,-19.3,1005.0,586.600,222.0,0,0


## <a id = 6 > Modelling Workflow

### <a id = 61 > Data Prep 

#### <a id = 612 > Normalizing numerical variables

Now that we have numerical features, can we feed it directly into a tree based model like Random Forest? Ye,s even without scaling or normalizing. Scaling has the effect of squeezing all data range into a tight range but is not required for Tree-based ML models

#### <a id='613'>Feature - Target Split</a> 

In [29]:
X = df.drop(target_class_name, axis=1)
y = df[target_class_name]

#### <a id='614'>Train-Test Split</a> 

In [17]:
# # split into train and test set 
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# # Note optionally convert the pandas dataframe into a numpy array using to_numpy if you have a big data
# # and want to model faster. Otherwise it doesnt matter which data structure you use

### <a id='62'>Estimate of baseline accuracy - Median prediction </a> 

In [22]:
# !pip install scikit-learn -U

    100% |████████████████████████████████| 7.1MB 5.3MB/s eta 0:00:01
    100% |████████████████████████████████| 296kB 31.4MB/s ta 0:00:01
  Found existing installation: scikit-learn 0.20.3
    Uninstalling scikit-learn-0.20.3:
      Successfully uninstalled scikit-learn-0.20.3


In [26]:
# Calculate root mean squared error
def rmse_metric(actual, predicted):
    sum_error = 0.0
    for i in range(len(actual)):
        prediction_error = predicted[i] - actual[i]
        sum_error += (prediction_error ** 2)
        mean_error = sum_error / float(len(actual))
    return np.sqrt(mean_error)

In [33]:
from sklearn.dummy import DummyRegressor

dummy_regr = DummyRegressor(strategy="median")
dummy_regr.fit(X, y)

y_pred = dummy_regr.predict(X)

rmse_metric(y.values, y_pred)



-0.08428390503568517


112.47855169938535

In [36]:
sample_df = pd.read_csv('input/' + sample_submission_file_name, index_col='ID')
sample_df.head()

,electricity_consumption
ID,
552,0
553,0
554,0
555,0
556,0


In [46]:
# Overwrite the sample submission file (assuming that the ID order is the same)
# list(test_df.index) == list(sample_df.index)
baseline_submission_df = sample_df.copy()
baseline_submission_df['electricity_consumption'] = dummy_regr.predict(test_df)
baseline_submission_df.to_csv('baseline_submission1.csv',index=True)

In [47]:
# Median solution gave 131 on the Public leaderboard